# BATTLE OF NEIGHBOURHOODS

This notebook is mainly for working on coursera capstone project, Given a city I will segment it into different neighbourhoods using the geographical coordinates of the center of each neighbourhood

Using a combination of location data and Machine Learning we will group the neighbourhoods into clusters.

We will be using Foursquare API as the location data provider

In [130]:
import pandas as pd
import numpy as np

In [131]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Exploring and Clustering the neighborhoods in Toronto.

Using the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [132]:
import requests # for sending http requests
from bs4 import BeautifulSoup # used for parsing HTML and XML documents


In [133]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#using requests.get(url).text will ask a website for the HTML information and teturns the same

In [ ]:
soup = BeautifulSoup(wiki_url,'lxml')
#print(soup.prettify()) # prettify will enable us to have a look at the how the tags are nested in the file we have parsed.

We need to find where to start with!! from the above tags. If closely observed we will find a table class under which we have all the information of the Postal codes and Neghborhood details. this is under the class 'wikitable sortable', so we will have to find this class in the script.

In [135]:
find_table = soup.find('table',{'class':'wikitable sortable'})

Again checking closely we can see that all the information lies between 'tr' tags and every 'td' is our content values.

In [136]:
rowValues = []

In [137]:
for tags in range(len(find_table.find_all('td'))):
    rowVal = find_table.find_all('td')[tags].get_text()
    rowValues.append(rowVal)

In [138]:
# creating a list for separate columns which can used as pandas column values
PostcodeList = []
BoroughList = []
NeighbourhoodList = []

In [139]:
for postcode in range(0, len(rowValues),3):
    PostcodeList.append(rowValues[postcode])
    
for borough in range(1, len(rowValues),3):
    BoroughList.append(rowValues[borough])
    
for neighbourhood in range(2, len(rowValues),3):
    NeighbourhoodList.append(rowValues[neighbourhood])    

In [140]:
import pandas as pd

tableDF = pd.DataFrame()
tableDF['Postcode'] = PostcodeList
tableDF['Borough'] = BoroughList
tableDF['Neighbourhood'] = NeighbourhoodList

In [141]:
#replacing \n in the Neighbourhood Column
tableDF = tableDF.replace('\n','',regex = True)

In [142]:
tableDF.shape

(287, 3)

Removing the rows  with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [143]:
#Removing the rows with a borough that is Not assigned.
for i in range (len(tableDF['Borough'])):
    try:
        if tableDF['Borough'][i] == 'Not assigned':
            tableDF = tableDF.drop(i)
    except KeyError:
        pass
    
tableDF = tableDF.reset_index(drop = True)  

In [144]:
tableDF.shape

(210, 3)

In [145]:
tableDF.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [146]:
#f a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for i in range (len(tableDF['Neighbourhood'])):
    try:
        if tableDF['Neighbourhood'][i] == 'Not assigned':
            tableDF['Neighbourhood'][i] == tableDF['Borough'][i]
    
    except KeyError:
        pass
        
tableDF = tableDF.reset_index(drop = True)       

In [147]:
tableDF.shape

(210, 3)

In [158]:
tableDF.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [167]:
tableDF_final = tableDF.groupby('Postcode',as_index=False,sort=False).agg({
    'Borough' : lambda x: x.max(),
    'Neighbourhood': lambda x: ', '.join(x)
    
    
})

since we are using group

In [168]:
tableDF_final.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [169]:
tableDF_final.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [172]:
!pip install geocoder

     |████████████████████████████████| 102kB 13.8MB/s ta 0:00:01


In [ ]:
import geocoder # import geocoder

g = geocoder.google('Mountain View, CA')

g.latlng
